In [1]:
import numpy

import copy  # to copy list of lists

def gauss(a, b):
    a = numpy.array(a, copy=True, dtype='float64')  # to copy internal lists too
    b = numpy.array(b, copy=True, dtype='float64')
    n = len(a)
    
    def forward():
        for i in range(n):
            for j in range(i+1, n):
                t = (a[j][i] / a[i][i])
                a[j] -= a[i] * t
                b[j] -= b[i] * t


    def backward():
        print(a, b)
        x = numpy.zeros(len(b))
        for i in range(n-1, -1, -1):
            x[i] = (b[i] - numpy.sum(a[i][i+1:] * x[i+1:])) / a[i][i]
        return x

    forward()
    return backward()


a = numpy.array([
    [1,2,1],
    [3,2,3],
    [1,0,0]
])

b = numpy.array([5,6,7])

x = gauss(a, b)
print("X =", x)
print("A * X =", numpy.matmul(a, x))

[[ 1.  2.  1.]
 [ 0. -4.  0.]
 [ 0.  0. -1.]] [ 5.  -9.   6.5]
X = [ 7.    2.25 -6.5 ]
A * X = [5. 6. 7.]


In [4]:
import numpy
from numba import njit, prange
import numpy.random

import copy  # to copy list of lists

@njit(fastmath=True)
def gauss_jit(a1 : 'numpy.ndarray', b1: 'numpy.ndarray') -> 'numpy.ndarray':
    a = numpy.copy(a1)
    b = numpy.copy(b1)
    n = len(a)
    
    def forward(a, b):
        for i in range(n):
            for j in range(i+1, n):
                t = (a[j][i] / a[i][i])
                a[j] = a[j] - a[i] * t
                b[j] -= b[i] * t


    def backward(a, b):
        x = numpy.zeros(len(b))
        for i in range(n-1, -1, -1):
            x[i] = (b[i] - numpy.sum(a[i][i+1:] * x[i+1:])) / a[i][i]
        return x

    forward(a, b)
    return backward(a, b)

N = 100
a = numpy.random.rand(N, N)

b = numpy.random.rand(N)

%timeit gauss(a, b)
%timeit gauss_jit(a, b)



20.9 ms ± 409 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
891 µs ± 23 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
